In [28]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from rdkit import Chem

# Load the original data with SMILES
url = 'https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/delaney-processed.csv'
df = pd.read_csv(url)

# Function to convert molecules to graph data objects
def mol_to_graph(smiles, y=None):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    # Get atom features
    num_atoms = mol.GetNumAtoms()
    
    # Simple atom features: atomic number and aromatic flag
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append([
            atom.GetAtomicNum(),
            1 if atom.GetIsAromatic() else 0,
            atom.GetTotalDegree(),
            atom.GetFormalCharge(),
            atom.GetTotalNumHs()
        ])
    
    x = torch.tensor(atom_features, dtype=torch.float)
    
    # Get edge indices (bonds)
    edge_indices = []
    edge_features = []
    
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        bond_type = bond.GetBondTypeAsDouble()
        
        # Add both directions for undirected graph
        edge_indices.append([i, j])
        edge_indices.append([j, i])

        #Add same bond features for both directions
        edge_features.append([bond_type])
        edge_features.append([bond_type])
        
    if len(edge_indices) == 0:  # Handle molecules with no bonds
        edge_index = torch.zeros((2, 0), dtype=torch.long)
        edge_attr = torch.zeros((0,1), dtype=torch.float)
    else:
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t()
        edge_attr = torch.tensor(edge_features,dtype=torch.float)
    
    # Create data object
    data = Data(x=x, edge_index=edge_index,edge_attr=edge_attr)
    
    if y is not None:
        data.y = torch.tensor([y], dtype=torch.float)
        
    return data

# Convert SMILES to graph objects with solubility values
graph_data_list = []
target_column = 'measured log solubility in mols per litre'

for i, row in df.iterrows():
    smiles = row['smiles']
    y = row[target_column]
    data = mol_to_graph(smiles, y)
    if data is not None:
        graph_data_list.append(data)

# Split data
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(graph_data_list, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [29]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, GATConv, global_mean_pool, global_add_pool
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score

class enhancedGNN(torch.nn.Module):
    def __init__(self,num_features=5,hidden_dim=64):
        super(enhancedGNN, self).__init__()

        #first GC layer
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)

        #second GC Layer
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)

        #third GC Layer
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)

        #Fully connected layers
        self.fc1 = nn.Linear(hidden_dim, 32)
        self.fc2 = nn.Linear(32,1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        # First conv block
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)

        # Second conv block
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        
        # Third conv block
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.relu(x)
        
        # Global pooling to get graph-level representation
        x = global_mean_pool(x, batch)
        
        # Fully connected layers
        x = F.dropout(x,p=0.2, training=self.training)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x.view(-1)

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = enhancedGNN(num_features=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor = 0.5, patience=10, min_lr=1e-6)
criterion = nn.MSELoss()

# Training function
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        #Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

# Early stopping to prevent overfitting
def train_with_early_stopping(patience=20):
    best_loss = float('inf')
    best_epoch = 0
    best_model_state = None
    epochs_no_improve = 0


    for epoch in range(200): 
        train_loss = train()
        test_loss, _, _ = test(test_loader)

        #Learning rate scheduling
        scheduler.step(test_loss)
    
        if (epoch + 1) % 10 == 0:
            print(f'Epoch: {epoch+1:03d}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

        #check if this is the best
        if test_loss < best_loss:
            best_loss = test_loss
            best_epoch = epoch
            best_model_state = copy.deepcopy(model.state_dict())
            epoch_no_improve = 0
        else:
            epoch_no_improve +=1

        #early stopping check
        if epoch_no_improve >= patience:
            print(f'Early stopping triggered after epoch {epoch+1}')
            print(f'Best model was at epoch {best_epoch+1} with test loss {best_loss:.4f}')
            break

    model.load_state_dict(best_model_state)
    return best_epoch, best_loss

import copy
best_epoch, best_loss = train_with_early_stopping(patience=20)

# Final evaluation
_, test_pred, test_actual = test(test_loader)
mse = mean_squared_error(test_actual, test_pred)
r2 = r2_score(test_actual, test_pred)

print(f'Test MSE: {mse:.4f}')
print(f'Test RMSE: {np.sqrt(mse):.4f}')
print(f'Test R²: {r2:.4f}')

/home/ria/Documents/YRP_iiit/Mac_Lea/molecular_property_prediction/venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/ria/Documents/YRP_iiit/Mac_Lea/molecular_property_prediction/venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 010, Train Loss: 1.7918, Test Loss: 8.0245
Epoch: 020, Train Loss: 1.6039, Test Loss: 7.7406
Early stopping triggered after epoch 22
Best model was at epoch 2 with test loss 6.7360
Test MSE: 2.8759
Test RMSE: 1.6959
Test R²: 0.3916
